# Tema 7. PSI-BLAST

Cuando se secuencian los mRNA de un organismo y se ensamblan las lecturas, se genera un **transcriptoma**: la colección de secuencias de los genes expresados en los tejidos que formaban parte de la muestra. Muchos transcriptomas ensamblados pueden encontrarse en la *European Nucleotide Archive*. El análisis siguiente típicamente consiste en determinar la función de cada gen. Para ello, los tránscritos codificantes de proteína son traducidos y las secuencias proteicas se comparan con las de las bases de datos. Las bases de datos más útiles para identificar la función de los tránscritos son las que tienen mayor información funcional: Pfam, Swissprot, KEGG, etc.

En esta práctica hemos tomado un transcriptoma del escarabajo *Ips typographus*, con número de acceso GACR01000000 en ENA. De los 14689 tránscritos, se han seleccionado 1361 que parecen codificar algún péptido de al menos 150 aminoácidos. Se ha realizado ya un primer BLASTP con las 1361 proteínas hipotéticas, y la mayor parte de ellas han encontrado alguna homología en la base de datos Swissprot.

En esta práctica nos vamos a fijar en las 57 de aquellas 1361 proteínas cuyo mejor resultado en el BLASTP contra Swissprot ha obtenido un valor E mayor o igual a 0.1 (pero no mayor que 1). El objetivo es determinar si mediante búsquedas de PSI-BLAST, más sensibles, logramos assignar estas proteïnas a algún grupo de proteínas remotamente parecidas que nos puedan dar alguna pista de su función.

El archivo `badHits.fa` es el archivo fasta con las 57 proteínas (hipotéticas) de *Ips typographus* seleccionadas.

In [ ]:
# Si no funciona esto, usa la consola para instalar blast.
system2(command = 'conda',
        args = c('install', '-c', 'bioconda', '-y', 'blast'),
        stdout = TRUE)

In [ ]:
system2(command = 'tar', args = c('-xzf', 'taxdb.tar.gz'), wait = TRUE)

In [ ]:
system2('psiblast',
        args = c('-query', 'badHits.fa',
                 '-db', 'swissprot',
                 '-out', 'psiblast.out',
                 '-outfmt', '7',
                 '-evalue', '1',
                 '-inclusion_ethresh', '0.7',
                 '-num_iterations', '5'))

## Ejercicios

1. Corrige si hace falta el código anterior y ejecútalo.
2. Revisa los resultados y determina si hay proteínas para las que todavía no hemos encontrado homologías.
3. Piensa y discute si los resultados del PSI-BLAST nos pueden ayudar a determinar la función de una proteína desconocida, o en qué casos.
4. Piensa y comprueba qué pasa (y para qué serviría) canviar los umbrales de valor E, `-evalue` y `-inclusion_ethresh`.

## Análisis individual
Hemos ejecutado el PSI-BLAST para las 57 proteínas de una vez, y los resultados aparecen todos en el mismo archivo. Sin embargo, puede ser más fácil procesar los resultados individuales de cada *query*. Podemos crear archivos fasta  para proteínas individuales desde R, mediante la función `write.fasta()` del paquete `seqinr`:

In [ ]:
library(seqinr)
library(Biostrings)

Las57 <- read.fasta('badHits.fa')
names(Las57)
write.fasta(Las57[['GACR01008940.1']],
            names = 'GACR01008940.1',
            file.out = 'GACR01008940.fa')

El PSI-BLAST elabora, después de cada ronda de búsqueda, una *position-specific scoring matrix* (PSSM), que resume la información de haber alineado las proteínas homólogas encontradas. De esta forma, la búsqueda es más sensible que en un BLASTP convencional. Podemos guardar la PSSM de la última ronda de PSI-BLAST para realizar con ella una búsqueda manual después:

In [ ]:
system2('psiblast',
        args = c('-query', 'GACR01008940.fa',
                 '-db', 'swissprot',
                 '-out', 'z1.out',
                 '-outfmt', '7',
                 '-evalue', '1',
                 '-inclusion_ethresh', '0.7',
                 '-num_iterations', '4',
                 '-save_pssm_after_last_round',
                 '-out_pssm', 'GACR01008940.pssm'))



Comprueba que ha aparecido el archivo `GACR01008940.pssm`, que ahora podemos usar como *query*:

In [ ]:
system2('psiblast',
        args = c('-in_pssm', 'GACR01008940.pssm',
                 '-db', 'swissprot',
                 '-out', 'last.out',
                 '-outfmt', '7',
                 '-evalue', '1'))

Una ventaja de realizar un último paso de PSI-BLAST con la PSSM es que obtenemos una tabla de resultados única, y no una para cada iteración. Esto facilita cargar los resultados finales en la sesión de R:

In [ ]:
resultado <- read.table('last.out',
                        col.names = c('query','subject','identity','length','mismatch',
                                      'gaps','qstart','qend','sstart','send','evalue','bitscore'),
                        stringsAsFactors = FALSE)
plot(resultado$evalue, type = 'b', log = 'y')


In [ ]:
plot(c(1,380), range(resultado$evalue), type = 'n', ylab = 'Valor E', xlab = 'Aminoácidos', log = 'y')
segments(resultado$qstart, resultado$evalue, resultado$qend, resultado$evalue)

## Ejercicios

1. Si queda tiempo, repite este tipo de análisis para las proteínas que te interesen.
2. ¿Cómo podrías repetir el proceso de forma automática con otras proteínas?